In [1]:
%run 01_loading_model.ipynb

In [196]:
our = ['isopentenyl', 'prenyl diphosphate', 'farnesyl', 'geranyl']
our_reactions = list()
translate = dict()
for name in our:
    for m in model.metabolites.query(r'^' + name, 'name'):
        print(m.id, "|", m.name, m.compartment)
        if m.compartment == 'c':
            our_reactions.append(m.reactions)
            translate[m.id] = m.name

s_0943[c] | isopentenyl diphosphate [cytoplasm] c
s_0944[lp] | isopentenyl diphosphate [lipid particle] lp
s_0945[m] | isopentenyl diphosphate [mitochondrion] m
s_1376[c] | prenyl diphosphate(3-) [cytoplasm] c
s_0190[c] | farnesyl diphosphate [cytoplasm] c
s_0191[m] | farnesyl diphosphate [mitochondrion] m
s_0692[lp] | farnesyl diphosphate [lipid particle] lp
s_0189[c] | geranylgeranyl diphosphate [cytoplasm] c
s_0745[c] | geranyl diphosphate [cytoplasm] c
s_0746[lp] | geranylgeranyl diphosphate [lipid particle] lp


In [198]:
react_stoich = list()
for elem in our_reactions:
    for r in elem:
        print(r, '| NAME:', r.name)
        stoich = str(r).split(' ')

r_1182: s_0943[c] <=> s_0945[m] | NAME: isopentenyl diphosphate transport
r_0355: s_0943[c] + s_1376[c] --> s_0633[c] + s_0745[c] | NAME: dimethylallyltranstransferase
r_0461: s_0189[c] + s_0943[c] --> s_0633[c] + s_1311[c] | NAME: geranylgeranyltranstransferase
r_0667: s_0943[c] <=> s_1376[c] | NAME: isopentenyl-diphosphate D-isomerase
r_0462: s_0745[c] + s_0943[c] --> s_0190[c] + s_0633[c] | NAME: geranyltranstransferase
r_0739: s_0018[c] + s_0434[c] --> s_0394[c] + s_0456[c] + s_0943[c] + s_1322[c] | NAME: mevalonate pyrophoshate decarboxylase
IDI: s_0943[c] --> dmapp | NAME: 
r_0373: s_0190[c] + s_0943[c] --> s_0189[c] + s_0633[c] | NAME: farnesyltranstransferase
GGPPS/CrtE: dmapp + s_0943[c] --> ggpp + 3 s_0633[c] | NAME: 
r_0667: s_0943[c] <=> s_1376[c] | NAME: isopentenyl-diphosphate D-isomerase
r_0355: s_0943[c] + s_1376[c] --> s_0633[c] + s_0745[c] | NAME: dimethylallyltranstransferase
r_0462: s_0745[c] + s_0943[c] --> s_0190[c] + s_0633[c] | NAME: geranyltranstransferase
r_46

In [206]:
model.reactions.get_by_id('r_0355')

Reaction identifier,r_0355
Name,dimethylallyltranstransferase
Memory address,0x07f8811de9e50
Stoichiometry,s_0943[c] + s_1376[c] --> s_0633[c] + s_0745[c] isopentenyl diphosphate [cytoplasm] + prenyl diphosphate(3-) [cytoplasm] --> diphosphate [cytoplasm] + geranyl diphosphate [cytoplasm]
GPR,YJL167W
Lower bound,0.0
Upper bound,1000.0


In [205]:
model.reactions.get_by_id('r_0462')

Reaction identifier,r_0462
Name,geranyltranstransferase
Memory address,0x07f8811dbd940
Stoichiometry,s_0745[c] + s_0943[c] --> s_0190[c] + s_0633[c] geranyl diphosphate [cytoplasm] + isopentenyl diphosphate [cytoplasm] --> farnesyl diphosphate [cytoplasm] + diphosphate [cytoplasm]
GPR,YJL167W
Lower bound,0.0
Upper bound,1000.0


In [209]:
model.metabolites.get_by_id('s_0803[c]')

Metabolite identifier,s_0803[c]
Name,H2O [cytoplasm]
Memory address,0x07f882140fd30
Formula,H2O
Compartment,c
In 366 reaction(s),"r_4412, r_1722, r_4567, r_0013, r_0240, r_4407, r_0465, r_4160, r_0238, r_0524, r_0788, r_4569, r_3153, r_0954, r_3154, r_0703, r_4413, r_1723, r_4678, r_0328, r_0502, r_4583, r_0657, r_4571,..."


In [213]:
for m in model.metabolites.query('FAD', 'name'):
    print(m.id, m.name)

s_0687[c] FAD [cytoplasm]
s_0688[m] FAD [mitochondrion]
s_0689[c] FADH2 [cytoplasm]
s_0690[m] FADH2 [mitochondrion]


In [234]:

model.metabolites.get_by_id('s_0688[m]')

Metabolite identifier,s_0688[m]
Name,FAD [mitochondrion]
Memory address,0x07f882146bca0
Formula,C27H30N9O15P2
Compartment,m
In 4 reaction(s),"r_1790, r_0454, r_0940, r_0490"


In [238]:
model.reactions.get_by_id('r_0490')

Reaction identifier,r_0490
Name,glycerol-3-phosphate dehydrogenase (fad)
Memory address,0x07f8811d0edf0
Stoichiometry,s_0688[m] + s_0770[m] + s_0799[m] --> s_0632[m] + s_0690[m] FAD [mitochondrion] + glycerol 3-phosphate [mitochondrion] + H+ [mitochondrion] --> dihydroxyacetone phosphate [mitochondrion] + FADH2 [mitochondrion]
GPR,YIL155C
Lower bound,0.0
Upper bound,1000.0


In [55]:
with model:
    model.objective = model.reactions.CrtI
    lycopene_production = model.optimize().objective_value
    print('Maximum productivity of lycopene =', lycopene_production, 'mmol/gDW*h')
    
    glc_uptake_flux = model.reactions.r_1714.flux
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc')

Maximum productivity of lycopene = 0.17793200061522896 mmol/gDW*h
Maximum theoretical yield = 0.17793200061522896 mmol-lyco/mmol-glc


We can then try to increase the glucose uptake as well as maximize lycopene production

In [83]:
for i in model.medium.items():
    print(i[0], '|', model.reactions.get_by_id(i[0]).name, '\t flux = \t', i[1])

r_1654 | ammonium exchange 	 flux = 	 1000.0
r_1714 | D-glucose exchange 	 flux = 	 1.0
r_1832 | H+ exchange 	 flux = 	 1000.0
r_1861 | iron(2+) exchange 	 flux = 	 1000.0
r_1992 | oxygen exchange 	 flux = 	 1000.0
r_2005 | phosphate exchange 	 flux = 	 1000.0
r_2020 | potassium exchange 	 flux = 	 1000.0
r_2049 | sodium exchange 	 flux = 	 1000.0
r_2060 | sulphate exchange 	 flux = 	 1000.0
r_2100 | water exchange 	 flux = 	 1000.0
r_4593 | chloride exchange 	 flux = 	 1000.0
r_4594 | Cu2(+) exchange 	 flux = 	 1000.0
r_4595 | Mn(2+) exchange 	 flux = 	 1000.0
r_4596 | Zn(2+) exchange 	 flux = 	 1000.0
r_4597 | Mg(2+) exchange 	 flux = 	 1000.0
r_4600 | Ca(2+) exchange 	 flux = 	 1000.0


In [91]:
model.reactions.CrtI

Reaction identifier,CrtI
Name,
Memory address,0x07f882153e910
Stoichiometry,phyto + 4 s_1198[c] --> lyco + 4 s_1203[c] Phytoene + 4 NAD [cytoplasm] --> Lycopene + 4 NADH [cytoplasm]
GPR,
Lower bound,0.0
Upper bound,1000.0


In [90]:
medium = model.medium
for uptake_rate in (10, 20, 30, 100, 1000):
    print('Setting glucose uptake rate to:', uptake_rate)
    with model:
        medium['r_1714'] = uptake_rate
        model.medium = medium
        max_growth = model.optimize().objective_value
        print('Maximum theoretical growth rate:', max_growth, 'h-1 \n')
        
with model:
    medium['r_1714'] = 1000
    model.objective = model.reactions.CrtI
    lyco_prod = model.optimize().objective_value
    print('Maximum theoretical productivity of lycopene =', lyco_prod, 'mmol/gDW*h')
        

Setting glucose uptake rate to: 10
Maximum theoretical growth rate: 0.8637367930478762 h-1 

Setting glucose uptake rate to: 20
Maximum theoretical growth rate: 1.7303913341660815 h-1 

Setting glucose uptake rate to: 30
Maximum theoretical growth rate: 2.597045875284299 h-1 

Setting glucose uptake rate to: 100
Maximum theoretical growth rate: 8.663627663111763 h-1 

Setting glucose uptake rate to: 1000
Maximum theoretical growth rate: 19.816830259464354 h-1 

Maximum theoretical productivity of lycopene = 0.17793200061522835 mmol/gDW*h


<div class="alert alert-danger" role="alert">
        NO change in production of lycopene when glucose uptake maximized!
</div>


In [114]:
for r in model.reactions.query('L-', 'name'):
    print(r.id, r.name)

r_0045 3-hydroxy-L-kynurenine hydrolase
r_0069 4-hydroxy-L-threonine synthase
r_0672 L-1-pyrroline-3-hydroxy-5-carboxylate dehydrogenase
r_0673 L-4-hydroxyglutamate semialdehyde dehydrogenase
r_0674 L-alanine transaminase
r_0675 L-allo-threonine aldolase
r_0676 L-allo-threonine dehydrogenase
r_0678 L-aminoadipate-semialdehyde dehydrogenase (NADPH)
r_0679 L-asparaginase
r_0680 L-asparaginase
r_0681 L-erythro-4-hydroxyglutamate:2-oxoglutarate aminotransferase
r_0682 L-erythro-4-hydroxyglutamate:2-oxoglutarate aminotransferase
r_0683 L-erythro-4-hydroxyglutamate:2-oxoglutarate aminotransferase
r_0687 L-hydroxyproline reductase (NADP)
r_0688 L-lactaldehyde:NADP+ 1-oxidoreductase
r_0689 L-serine deaminase
r_0690 L-serine dehydrogenase
r_0691 L-sorbitol dehydrogenase (L-sorbose-producing)
r_0692 L-threonine deaminase
r_0693 L-threonine deaminase
r_0694 L-tryptophan:oxygen 2,3-oxidoreductase (decyclizing)
r_0695 L-tyrosine N-formyltransferase
r_0762 N-formyl-L-kynurenine amidohydrolase
r_0815

In [115]:
model.reactions[1]

Reaction identifier,r_0002
Name,(R)-lactate:ferricytochrome-c 2-oxidoreductase
Memory address,0x07f881200fa90
Stoichiometry,s_0027[m] + 2.0 s_0709[m] --> 2.0 s_0710[m] + s_1401[m] (R)-lactate [mitochondrion] + 2.0 ferricytochrome c [mitochondrion] --> 2.0 ferrocytochrome c [mitochondrion] + pyruvate [mitochondrion]
GPR,( YDL178W and YEL039C ) or ( YDL178W and YJR048W )
Lower bound,0.0
Upper bound,1000.0
